In [2]:
##
##Script to get List of categories available in Yelp that list restraurants in ranking
##
import pandas as pd
import numpy as np
#for creating geo coordinate combinations
import itertools  as it
from yelpapi import YelpAPI
import requests     # how python goes onto the internet!
import re           # regex
from bs4 import BeautifulSoup # a python HTML parser (version 3)
from urllib2 import Request, urlopen
import json
from pandas.io.json import json_normalize
%matplotlib inline
from pprint import pprint
import urllib
import time
import datetime

In [3]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [4]:
def met(url):
    r = requests.get(url)
    b = BeautifulSoup(r.text, 'html.parser') # try this line instead if you have problems
    #Make a dictionary of description table (tombstone)
    dl = b.find_all("dl")
    dc = {}
    for i in dl:
        dd = []
        for d in i.find_all("dd"):
            dd += [d.text.replace('\n', '').replace('\r', '').replace('\u', '')]
        dc[i.find_all("dt")[0].text.split(":")[:1][0]] = dd
    
    #dc is the top table
    #creating tc to get rest of the detail about the artwork
    h = [span.get_text() for span in b.find_all("header",{"class":"accordion__header"})][1:]
    d = [span.get_text() for span in b.find_all("div",{"class":"accordion__content ng-hide"})]
    #for i in h:i.replace(' ', '')
    h = [i.replace('\n', '').replace('\r', '').replace('  ','') for i in h]
    d = [i.replace('\n', '').replace('\r', '').replace('  ','') for i in d]
    tc = dict(zip(h, d))
    met = dict(dc, **tc)
    
    #Add new columns
    met['SourceLink'] = url
    #met['ImageURL'] = find_between(str(b.findAll('div', attrs={'class' : 'collection-results'})[0].find_all('img')[0].get('ng-src')).replace("'", ''), '(', ')')
    #met['ImageURL2'] = (str(b.findAll('div', attrs={'class' : 'collection-results'})[0].find_all('img')[0].get('src')))
    
    if (str(b.findAll('div', attrs={'class' : 'collection-results'})[0].find_all('img')[0].get('src'))) == 'None': 
        link = find_between(str(b.findAll('a', attrs={'name' : '#collectionImage'})[0].find_all('img')[0].get('ng-src')).replace("'", ''), '(', ')')
    else: link = (str(b.findAll('div', attrs={'class' : 'collection-results'})[0].find_all('img')[0].get('src')))

    if link.split('/')[-1] != 'NoImageAvailableIcon.png': 
        met['ImageURL'] = link
    if link.split('/')[-1] != 'NoImageAvailableIcon.png': 
        met['LocalURL'] = '../data/MET/MET' +  link.split('/')[-1].replace(' ', '')
    
    #if met['ImageURL2'] == 'None': 
    #    link = met['ImageURL']
    #else: link = met['ImageURL2']
    
    #print link
    #print '../data/MET/MET' 
    #print link.split('/')[-1]
    if link != 'None' and link.split('/')[-1] != 'NoImageAvailableIcon.png':
        urllib.urlretrieve(link, filename = '../data/MET/MET' +  link.split('/')[-1])
    
    met['gallery'] = [span.get_text() for span in b.find_all("span",{"class":"card__meta-gallery"})][0].split(":",1)[1]
    met['Title'] = b.findAll('h1', attrs={'class' : 'collection-details__object-title'})[0].text
    met = pd.DataFrame(met)  
    nxt_url = 'http://www.metmuseum.org' + b.findAll('a', attrs={'class':'collection-results__nav collection-results__nav-next'})[0]['href']
    return met, nxt_url;

In [1030]:
url = 'http://www.metmuseum.org/art/collection/search/71636?pos=16&pg=1&rpp=20&offset=0&rndkey=20161127&ft=*&deptids=6'
temp = met(url)
a = temp[0]

In [972]:
temp[0]

,Accession Number,Classification,Credit Line,Culture,Date,Dimensions,ImageURL,LocalURL,Medium,Period,SourceLink,Timeline of Art History (2000-present),Title,gallery
0,20.148.641,Textiles-Woven,"Gift of Kihei Hattori, in memory of his father...",Japan,18th century,4 7/8 in. × 2 in. (12.4 × 5.1 cm),/content/img/placeholders/NoImageAvailableIcon...,../data/MET/METNoImageAvailableIcon.png,Silk / Compound weave,Edo period (1615–1868),http://www.metmuseum.org/art/collection/search...,"Kihei Hattori , Tokyo (until 1920)",Piece,Gallery 599


In [1091]:
start = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')
print start
url = 'http://www.metmuseum.org/art/collection/search/38780?pos=1&pg=1&rpp=20&offset=0&rndkey=20161127&ft=*&deptids=6'    
i = 1
a = pd.DataFrame({})
while i<= 50:
    c = met(url)
    a = a.append(c[0])
    url = c[1]
    i+=1
    time.sleep(time.localtime(time.time())[5]/15.0);
end = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')
print end

2016-11-27 17:58:41
2016-11-27 18:01:33


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [979]:
a.to_csv('../data/met_asianart.csv',encoding='utf-8')

In [1022]:
a

,Accession Number,Artist,Classification,Credit Line,Culture,Date,Dimensions,ImageURL,LocalURL,Medium,MetPublications,Period,Provenance,"Signatures, Inscriptions, and Markings",SourceLink,Timeline of Art History (2000-present),Title,gallery
0,13.96.8,NaN,Sculpture,"Rogers Fund, 1913",Pakistan (ancient region of Gandhara),3rd–5th century,H. 5 1/8 in. (13 cm),http://images.metmuseum.org/CRDImages/as/web-l...,../data/MET/MET16446.jpg,Stone,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search...,"[ Colonel M. C. Cooke-Collis , England, ca. 18...",Head of a Bodhisattva,Gallery 235
0,24.102.5,NaN,Metalwork,"Rogers Fund, 1924",China,NaN,H. 2 in. (5.1 cm); W. 1 in. (2.5 cm),http://images.metmuseum.org/CRDImages/as/web-t...,../data/MET/METDP123377.jpg,Gilt bronze,NaN,Tang dynasty (618–907),NaN,NaN,http://www.metmuseum.org/art/collection/search...,"TimelinesCentral and North Asia, 500–1000 A.D....",Ornament for Girdle,Gallery 207
0,10.211.1282,NaN,Netsuke,"Gift of Mrs. Russell Sage, 1910",Japan,19th century,H. 5/8 in. (1.6 cm); Diam. 1 1/2 in. (3.8 cm),http://images.metmuseum.org/CRDImages/as/web-t...,../data/MET/MET24_102_5_O.JPG,Ivory,NaN,NaN,NaN,Signature: Mei--(?),http://www.metmuseum.org/art/collection/search...,"TimelinesJapan, 1800–1900 A.D.",Netsuke,Gallery 242
0,1998.485.8,Kiyomizu Rokubei VI (1901–1980),Ceramics,"Gift of Mr. and Mrs. Yukikazu Iwasa, in memory...",Japan,20th century,Gr. Diam. 7 in. (17.8 cm),http://images.metmuseum.org/CRDImages/as/web-t...,../data/MET/METLC-10_211_1282-001.jpg,Molded pottery with overglaze enamels (Kyoto w...,NaN,Shōwa period (1926–89),NaN,NaN,http://www.metmuseum.org/art/collection/search...,NaN,Dish,Not on view
0,13.101.1,NaN,Tomb Pottery,"Gift of John C. Ferguson, 1913",China,NaN,H. 23 1/4 in. (59.1 cm),http://images.metmuseum.org/CRDImages/as/web-t...,../data/MET/MET1998_485_8.jpg,Unglazed pottery,NaN,Tang dynasty (618–907),NaN,NaN,http://www.metmuseum.org/art/collection/search...,NaN,Mortuary figure,Gallery 204
0,20.148.640,NaN,Textiles-Woven,"Gift of Kihei Hattori, in memory of his father...",Japan,18th century,1 7/8 × 4 7/8 in. (4.8 × 12.4 cm),http://images.metmuseum.org/CRDImages/as/web-t...,../data/MET/MET16426.jpg,Silk / Compound weave,NaN,Edo period (1615–1868),NaN,NaN,http://www.metmuseum.org/art/collection/search...,"Kihei Hattori , Tokyo (until 1920)",Piece,Gallery 599
0,14.40.123,NaN,Ceramics,"Bequest of Benjamin Altman, 1913",China,NaN,H. 6 1/2 in. (16.5 cm),http://images.metmuseum.org/CRDImages/as/web-t...,../data/MET/METDP15302.jpg,Porcelain,NaN,"Qing dynasty (1644–1911), Qianlong period (173...",NaN,NaN,http://www.metmuseum.org/art/collection/search...,Marking: None,Vase,Gallery 200
0,02.18.271,NaN,Jade,"Gift of Heber R. Bishop, 1902",America (Alaska),NaN,H. 4 15/16 in. (12.6 cm); W. 11/16 in. (1.8 cm...,http://images.metmuseum.org/CRDImages/as/web-t...,../data/MET/MET20852.jpg,Nephrite,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search...,"Heber R. Bishop , New York (until 1902)",Knife-sharpener,Not on view
0,JP229,"Hishikawa Moronobu (Japanese, died 1694)",Prints,"Rogers Fund, 1918",Japan,late 17th century,10 x 15 in. (25.4 x 38.1 cm),http://images.metmuseum.org/CRDImages/as/web-t...,../data/MET/MET02_18_271Knife-Sharpener.JPG,Polychrome woodblock print; ink and color on p...,NaN,Edo period (1615–1868),NaN,NaN,http://www.metmuseum.org/art/collection/search...,Signature: None.From a folding albumonly at th...,The Twelfth Scene from Scenes of the Pleasure ...,Not on view
0,02.18.464,NaN,Seals,"Gift of Heber R. Bishop, 1902",China,18th century,H. 3 3/8 in. (8.5 cm); W. 4 3/16 in. (10.6 cm)...,http://images.metmuseum.org/CRDImages/as/web-t...,../data/MET/METDP124567.jpg,Jade (nephrite),NaN,Qing dynasty (1644–1911),"Heber R. Bishop , New York (until 1902)",NaN,http://www.metmuseum.org/art/collection/search...,"TimelinesCentral and North Asia, 1600–1800 A.D...",清中期 雙龍鈕玉印Seal with two dragons\r\n\r\n,Gallery 222


In [998]:
a = met('http://www.metmuseum.org/art/collection/search/67603?pos=25&pg=2&rpp=20&offset=0&rndkey=20161127&ft=*&deptids=6')
a[0]

/content/img/placeholders/NoImageAvailableIcon.png


,Accession Number,Classification,Credit Line,Culture,Date,Dimensions,ImageURL,LocalURL,Medium,SourceLink,Timeline of Art History (2000-present),Title,gallery
0,96.14.1890,Textiles-Woven,"Gift of Mr. and Mrs. H. O. Havemeyer, 1896",Japan,18th–19th century,3 1/2 x 4 3/4 in. (8.89 x 12.07 cm),/content/img/placeholders/NoImageAvailableIcon...,../data/MET/METNoImageAvailableIcon.png,Silk,http://www.metmuseum.org/art/collection/search...,"Inscription: woven, unread",Piece,Gallery 599


In [1001]:
url = 'http://www.metmuseum.org/art/collection/search/67603?pos=25&pg=2&rpp=20&offset=0&rndkey=20161127&ft=*&deptids=6'
r = requests.get(url)
b = BeautifulSoup(r.text, 'html.parser') # try this line instead if you have problems


/content/img/placeholders/NoImageAvailableIcon.png


In [1018]:
if (str(b.findAll('div', attrs={'class' : 'collection-results'})[0].find_all('img')[0].get('src'))) not in ['None', 'False']: 
    link = find_between(str(b.findAll('div', attrs={'class' : 'collection-results'})[0].find_all('img')[0].get('ng-src')).replace("'", ''), '(', ')')
else: link = (str(b.findAll('div', attrs={'class' : 'collection-results'})[0].find_all('img')[0].get('src')))
print link
print (str(b.findAll('a', attrs={'name' : '#collectionImage'})[0].find_all('img')[0].get('ng-src')))
print find_between(str(b.findAll('a', attrs={'name' : '#collectionImage'})[0].find_all('img')[0].get('ng-src')).replace("'", ''), '(', ')')
    


{{currImage.selectedOrDefaultPreview('http://images.metmuseum.org/CRDImages/as/web-large/DP13151.jpg')}}
http://images.metmuseum.org/CRDImages/as/web-large/DP13151.jpg


In [1064]:
import datetime
datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d %H:%M:%S')

'2016-11-27 17:54:23'

In [1090]:
time.localtime(time.time())[5]/45.0

1.1777777777777778

# Scraping VictoriaAlbert

In [5]:
url = 'http://collections.vam.ac.uk/search/?listing_type=imagetext&offset=0&limit=45&narrow=1&extrasearch=&q=india&commit=Search&quality=0&objectnamesearch=&placesearch=&after=&after-adbc=AD&before=&before-adbc=AD&namesearch=&materialsearch=&mnsearch=&locationsearch='
r = requests.get(url)
b = BeautifulSoup(r.text, 'html.parser') # try this line instead if you have problems

In [6]:
#'http://collections.vam.ac.uk/' 
x = b.find_all('div', {'id':'results_list'})[0].find_all('div', attrs = {'class': 'item-imagetext pull '}) #[0].find_all('a')[0].get('href')
results = []
for i in x:
    x = 'http://collections.vam.ac.uk/' + i.find_all('a')[0].get('href')
    results += [x]
print results

[u'http://collections.vam.ac.uk//item/O82552/sita-painting-roy-jamini/', u'http://collections.vam.ac.uk//item/O72319/the-costumes-and-people-of-photographs-w-w-hooper/', u'http://collections.vam.ac.uk//item/O78357/relief-panel-panel/', u'http://collections.vam.ac.uk//item/O108360/buddhist-vihara-cave-ajanta-painting-simpson-william/', u'http://collections.vam.ac.uk//item/O140028/hampi-vijayanagar-bellary-district-virupaksha-photograph-greenlaw-alexander-john/', u'http://collections.vam.ac.uk//item/O82269/the-bull-nandi-in-the-painting-simpson-william/', u'http://collections.vam.ac.uk//item/O81457/an-ivory-writing-casket-inlaid-writing-case-unknown/', u'http://collections.vam.ac.uk//item/O82545/painting-roy-jamini/', u'http://collections.vam.ac.uk//item/O82551/jatayu-sita-and-ravana-painting-roy-jamini/', u'http://collections.vam.ac.uk//item/O130662/the-kulayana-mundapam-photograph-tripe-linnaeus/', u'http://collections.vam.ac.uk//item/O130660/side-collonnade-in-the-muroothappa-photogra

In [9]:
url = results[0]
b = BeautifulSoup(requests.get(url).text, 'html.parser')

In [8]:
#title
Title = b.find_all('h1')[0].text

#imageURL
ImageURL = b.find_all('div', {'class': 'item-image'})[0].find_all('a')[1]['href']

#basic details dictionary
h = b.find_all('div', attrs = {'id': 'basic-details'})[0].find_all('strong')
d = b.find_all('div', attrs = {'id': 'basic-details'})[0].find_all('p')
#for d in b.find_all('div', attrs = {'id': 'basic-details'})[0].find_all('strong'):
#    print d.text.replace(':', '')
h = [i.text.replace(':', '') for i in h]
d = [i.text.replace(':', '')for i in d]
bd = dict(zip(h, d))


In [112]:
summary = b.find_all('div', {'id': 'pane-summary'})[0].text

u"Jamini Roy (1887-1972) was one of the most important artists of the modern period in India, drawing on the popular and folk traditions of rural Bengal for his inspiration. He developed his own personal style which was characterised by bold lines and flat use of colour.\nThis image depicts a scene from the epic Hindu mythological poem, the Ramayana. Sita proves her innocence by undergoing a fire-ordeal. She is pictured sitting on a burning pyre amid fierce flames. The fire is stylised into sharp-edged flames. Her performance is witnessed by several onlookers including King Rama, Lakshman, two warriors and two monkeys. All the figures, have very large eyes, a characteristic feature of Jamini Roy's work. Jamini was a supporter of the Indian independence movement and his search for an authentic modern India was a response to this. Part of his search involved the restoration of the collaborative indigenous artisanal labour model. Thus, in his workshop he produced a range of stock motifs w

In [165]:
h = b.find_all('div', {'id': 'pane-more-information'})[0].find_all('h2')
p = b.find_all('div', {'id': 'pane-more-information'})[0].find_all('p')
h = [i.text for i in h][::-1]
p = [i.text for i in p][::-1]
#h = h[::-1]
minfo = dict(zip(h,p))
minfo

{u'Artist/maker': u'Height: 36.9 cm, Width: 43.8 cm, Height: 43.8 cm Conservation paper upon which card is mounted, Width: 53.1 cm Conservation paper upon which card is mounted',
 u'Bibliographic References (Citation, Note/Abstract, NAL no)': u"Partha Mitter, The Triumph of Modernism India' artists and the avant-garde 1922-1947, London, 2007\nW.G.Archer, India and Modern Art, London, 1959",
 u'Categories': u'Paintings; Hinduism; Myths & Legends; Bonita Trust Indian Paintings Cataloguing Project',
 u'Collection code': u'SSEA',
 u'Date': u'Signature in Hindi on right hand side.',
 u'Descriptive line': u'Painting, Sita in flames, by Jamini Roy or a member of his workshop, painting, opaque watercolour on cardboard, Kolkata, ca. 1940',
 u'Dimensions': u" In 1906 he entered the Calcutta School of Art and studied under Abanindranath Tagore, the pioneer and leading exponent of the Bengal School of Art. Abanindranath's tutorage secured Jamini's dexterity both in European and indigenous painting

NameError: name 'h' is not defined